# Get data and packages
Use the condat_python3 kernel, not the pyspark one

In [2]:
%%capture
%%bash
aws s3 cp --recursive s3://xdss-public-datasets/demos/taxi_parquet datasets/taxi_parquet

In [29]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os

instance_type = 'c5d2xlarge' # change this
results_bucket = f"s3://vaex-sagemaker-demo/benchmarks" # change this

name = 'spark'
data_path = 'datasets/taxi_parquet'
output_file = f'{name}_{instance_type}.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {
    'run':[],
    'duration': [],
    'task': []   
}

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks=benchmarks):
    return pd.DataFrame.from_dict(benchmarks, orient='index').T

def persist():
    gc.collect()
    get_results(benchmarks).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, **kwargs):    
    for i in range(2):
        start_time = time.time()
        ret = f(df, **kwargs)
        benchmarks['duration'].append(time.time()-start_time)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]
          
def add_nan(name):
    for i in range(2):
        benchmarks['duration'].append(np.nan)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]

          


!mkdir -p results
!mkdir -p datasets
print(f"We test every benchmark twice and save both results")

We test every benchmark twice and save both results


# Benchmark

In [ ]:
from pyspark import sql, SparkConf, SparkContext
import pyspark.sql.functions as f

conf = SparkConf().setAppName('Benchmarks')

# make sure you have enough memory for this
conf.setExecutorEnv('spark.executor.memory', '2g') 
conf.setExecutorEnv('spark.driver.memory', '12g')
sc = SparkContext(conf = conf)
sqlContext = sql.SQLContext(sc)
data = sqlContext.read.parquet(data_path)
print(f"size: {data.count()} with {len(data.columns)} columns")

In [31]:
# read/ open
def open_file(df=None):
    return sqlContext.read.parquet(data_path)

benchmark(open_file, df=data, name='read_file')

read_file took: 0.05755162239074707


0.05755162239074707

In [32]:
def count(df=None):
    return df.count()

benchmark(count, df=data, name='count')

count took: 0.31055116653442383


0.31055116653442383

In [33]:
def mean(df):
    return df.select(f.mean('fare_amount')).collect()

benchmark(mean, df=data, name='mean')

mean took: 2.3144118785858154


2.3144118785858154

In [34]:
def standard_deviation(df):
    return df.select(f.stddev('fare_amount')).collect()

benchmark(standard_deviation, df=data, name='standard deviation')

standard deviation took: 3.491633176803589


3.491633176803589

Unlike other techonlogies, vaex can return columns, or subset of values to explore lazely, 
but becouse many of the other tecnologies crashed at this point, we return a scalar instead.

In [35]:
def mean_of_sum(df):
    return df.select(f.mean(df['fare_amount'] + df['trip_distance'])).collect()

benchmark(mean_of_sum, df=data, name='sum columns mean')
add_nan('sum columns')

sum columns mean took: 3.6596174240112305
sum columns took: nan


nan

In [36]:
def mean_of_product(df):
    return df.select(f.mean(df['fare_amount'] * df['trip_distance'])).collect()

benchmark(mean_of_product, df=data, name='product columns mean')
add_nan('product columns')

product columns mean took: 3.7124032974243164
product columns took: nan


nan

In [37]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df['pickup_longitude']
    phi_1 = df['pickup_latitude']
    theta_2 = df['dropoff_longitude']
    phi_2 = df['dropoff_latitude']
    temp = (f.cos(theta_1)*np.pi/180) * (f.cos(theta_2)*np.pi/180) * (f.sin(phi_2-phi_1)/2*np.pi/180)**2
    expression = 2 * f.atan2(f.sqrt(temp), f.sqrt(1-temp))
    return df.select(f.mean(expression)).collect()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean')
add_nan('arithmetic operation')

arithmetic operation mean took: 92.70115447044373
arithmetic operation took: nan


nan

In [38]:
def value_counts(df):
    return df.select('passenger_count').distinct().collect()

benchmark(value_counts, df=data, name='value counts')

value counts took: 8.365249872207642


8.365249872207642

In [39]:
def groupby_statistics(df):
    ret = df.groupby('passenger_count').agg(
        f.mean('fare_amount'),
        f.stddev('fare_amount'),
        f.mean('tip_amount'),
        f.stddev('tip_amount')
    )
    ret.take(3)
    return ret

benchmark(groupby_statistics, df=data, name='groupby statistics')

groupby statistics took: 31.955618381500244


31.955618381500244

In [40]:
other = groupby_statistics(data)

In [41]:
def join(df, other):
    ret = df.join(other, on = 'passenger_count')
    ret.take(3)
    return ret
    
# benchmark(join, data, name='join', other=other)
add_nan('join')

join took: nan


nan

In [42]:
def join_count(df, other):
    return df.join(other, on = 'passenger_count').count()

benchmark(join_count, data, name='join count', other=other)

join count took: 296.01428627967834


296.01428627967834

## Filtered data

Dask is not build to run on filter data like you would normally, so we will apply the same strategy

In [43]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

Prepare filtered data and deleted 0 MB


In [44]:
def filter_data(df):
    expr_filter = (df.pickup_longitude > long_min)  & (df.pickup_longitude < long_max) & \
              (df.pickup_latitude > lat_min)    & (df.pickup_latitude < lat_max) & \
              (df.dropoff_longitude > long_min) & (df.dropoff_longitude < long_max) & \
              (df.dropoff_latitude > lat_min)   & (df.dropoff_latitude < lat_max)
    ret = df[expr_filter]
    ret.take(3) # evaluate the filter
    return ret
benchmark(filter_data, data, name='filter data')

filter data took: 0.0840616226196289


0.0840616226196289

In [45]:
filtered = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

cleaned 194 mb


In [ ]:
benchmark(filter_data, filtered, name='filtered count')
benchmark(mean, filtered, name='filtered mean')
benchmark(standard_deviation, filtered, name='filtered standard deviation')
benchmark(mean_of_sum, filtered, name ='filtered sum columns mean')
add_nan('filtered sum columns')
benchmark(mean_of_product, filtered, name ='filtered product columns mean')
add_nan('filtered product columns')
benchmark(mean_of_complicated_arithmetic_operation, filtered, name='filtered arithmetic operation mean')
add_nan('filtered arithmetic operation')
benchmark(value_counts, filtered, name ='filtered value counts')
benchmark(groupby_statistics, filtered, name='filtered groupby statistics')
other = groupby_statistics(filtered)
add_nan('filtered join')
benchmark(join_count, filtered, name='filtered join count', other=other)
print(name)
get_results(benchmarks)

In [48]:
print('Done spark')
get_results(benchmarks)

Done spark


,run,duration,task
0,1,0.0678499,read_file
1,2,0.0575516,read_file
2,1,0.283383,count
3,2,0.310551,count
4,1,2.32047,mean
5,2,2.31441,mean
6,1,3.5441,standard deviation
7,2,3.49163,standard deviation
8,1,3.61367,sum columns mean
9,2,3.65962,sum columns mean
